In [47]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import ImageFont, ImageDraw, Image
import random
import shutil
import unicodedata
from pathlib import Path
import os


In [48]:
new_folder_path = "NationalCardDigits"  # مسیر فولدر جدید برای ذخیره تصاویر با نام‌های تغییر یافته

if not os.path.exists(new_folder_path):
    os.makedirs(new_folder_path)

In [49]:
def generate_data(size):
    for digit in range(10):
        digit_data_dir = os.path.join('NationalCardDigits', str(digit))
        shutil.rmtree(digit_data_dir)
        if not os.path.exists(digit_data_dir):
            os.makedirs(digit_data_dir)
    persian_digits = ['۰', '۱', '۲', '۳', '۴', '۵', '۶', '۷', '۸', '۹']
    font = ImageFont.truetype('./font/Yekan.ttf', 32)
    template = Image.open("./BackgroundNationalcard/aa.jpg")
    template_w, template_h = template.size
    data_size = [0 for _ in range(10)]
    for idx in range(size):
        width, height = (48, 48)
        template_section_x = random.randint(template_w // 5, (template_w * 4) // 5)
        template_section_y = random.randint(template_h // 5, (template_h * 4) // 5)
        template_section_left = template_section_x - width // 2
        template_section_right = template_section_x + width // 2
        template_section_top = template_section_y - height // 2
        template_section_bottom = template_section_y + height // 2
        image = template.crop((template_section_left, template_section_top, template_section_right, template_section_bottom))
        label = random.randint(0, 9)
        digit = persian_digits[label]
        drawn = ImageDraw.Draw(image)

        digit_left, digit_top, digit_right, digit_bottom = drawn.textbbox([0, 0], digit, font=font)
        digit_width = digit_right - digit_left
        digit_height = digit_bottom - digit_top
        shift_x, shift_y = random.randint(-10, 10), random.randint(-10, 10)
        digit_x = (width - digit_width) // 2 + shift_x
        digit_y = (height - digit_height) // 2 + shift_y - 8

        drawn.text((digit_x, digit_y), digit, font=font, fill=(0, 0, 0))

        angle = random.randint(-10, 10)
        image = image.rotate(angle)
        image_array = np.array(image)
        gaussian_p = random.random()
        if gaussian_p < 0.5:
            blurred_image = cv2.GaussianBlur(image_array, (15, 15), 0)
            new_image = cv2.addWeighted(image_array, 0.5, blurred_image, 0.5, 0)
            image = Image.fromarray(new_image)
        noise_p = random.random()
        if noise_p < 0.5:
            mean = 0
            variance = 0.5
            sigma = np.sqrt(variance)
            noise = np.random.normal(mean, sigma, image_array.shape)
            noise = np.round(noise)
            noisy_image = image_array + noise
            noisy_image = noisy_image.astype(np.uint8)
            image = Image.fromarray(noisy_image)

        image_path = os.path.join(os.path.join('NationalCardDigits', str(label)), f'{data_size[label]}.jpg')
        image.save(image_path)
        data_size[label] += 1
    print('Dataset size:', data_size)

generate_data(10)

Dataset size: [1, 0, 1, 1, 1, 1, 1, 3, 0, 1]
